In [24]:
import pims
import numpy as np
import subprocess
import os
import time
import math
import matplotlib.pyplot as plt
import cv2

In [2]:
# methods to shorten videos

def cut_video(input_vid_dir, input_vid_name, start_seconds, end_seconds, clip_number, clip_path, last_or_no):
    """
    this method is super fast as it does not rely on encoding and deconding the video into wrappers and uses straight ffmpeg
    """
    
    # create name of video to be outputted
    output_name = input_vid_name[:-4] + 'short_{}.avi'.format("{:02d}".format(clip_number+1))
    output_path = os.path.join(clip_path, output_name)
    
    # create name of video to be outputted
    input_vid_path = os.path.join(input_vid_dir, input_vid_name)
    
    # format seconds into hh:mm:float(ss)
    start_seconds = time.strftime('%H:%M:%S', time.gmtime(start_seconds))
    end_seconds = time.strftime('%H:%M:%S', time.gmtime(end_seconds))
    
    # command format: 'ffmpeg -ss first_clip -i input_video_string -c copy -t second_clip output_video_string'
    
    if last_or_no == False:
        command = 'ffmpeg -ss {} -i {} -c copy -t {} {}'.format(start_seconds, input_vid_path, end_seconds, output_path)
        print(command)
    else:
        command = 'ffmpeg -ss {} -i {} -c copy {}'.format(start_seconds, input_vid_path, output_path)
        print(command)
    # this copies and clips the video. first number clips the first seconds, then clip everything that is seconds after that
    
    result = subprocess.run(command)


In [3]:
def get_length(filename):
    """
    returns length of the video in seconds as a float
    """
    result = subprocess.run(["ffprobe", "-v", "error", "-show_entries",
                             "format=duration", "-of",
                             "default=noprint_wrappers=1:nokey=1", filename],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT)
    return float(result.stdout)

In [4]:
def create_sliced_video(directory, video_for_slicing, division_number):
    """
    takes in video, spits out a folder containing shorter clips
    """
    video = os.path.join(directory, video_for_slicing)
    
    # create directoy for short clips
    clip_path = os.path.join(directory, video_for_slicing[:-4] + '_clips')
    os.mkdir(clip_path)
    
    # get length of video and length of clips based on the number of clips we want
    length = get_length(video)
    clip_length = length/division_number
    clip_length = math.floor(clip_length)
    
    # enumerate through clip numbers writing each one to the short clip directory
    for clip_number in range(division_number):
        
        start_seconds = clip_length*clip_number
        
        if clip_number == division_number-1:
            cut_video(directory, video_for_slicing, start_seconds, clip_length, clip_number, clip_path, True)
            
        else:
            cut_video(directory, video_for_slicing, start_seconds, clip_length, clip_number, clip_path, False)

In [5]:
create_sliced_video(r'C:\Users\BMLab21\Documents\CrabStreams', '2022-04-06_264.avi', 12)

ffmpeg -ss 00:00:00 -i C:\Users\BMLab21\Documents\CrabStreams\2022-04-06_264.avi -c copy -t 00:00:21 C:\Users\BMLab21\Documents\CrabStreams\2022-04-06_264_clips\2022-04-06_264short_01.avi
ffmpeg -ss 00:00:21 -i C:\Users\BMLab21\Documents\CrabStreams\2022-04-06_264.avi -c copy -t 00:00:21 C:\Users\BMLab21\Documents\CrabStreams\2022-04-06_264_clips\2022-04-06_264short_02.avi
ffmpeg -ss 00:00:42 -i C:\Users\BMLab21\Documents\CrabStreams\2022-04-06_264.avi -c copy -t 00:00:21 C:\Users\BMLab21\Documents\CrabStreams\2022-04-06_264_clips\2022-04-06_264short_03.avi
ffmpeg -ss 00:01:03 -i C:\Users\BMLab21\Documents\CrabStreams\2022-04-06_264.avi -c copy -t 00:00:21 C:\Users\BMLab21\Documents\CrabStreams\2022-04-06_264_clips\2022-04-06_264short_04.avi
ffmpeg -ss 00:01:24 -i C:\Users\BMLab21\Documents\CrabStreams\2022-04-06_264.avi -c copy -t 00:00:21 C:\Users\BMLab21\Documents\CrabStreams\2022-04-06_264_clips\2022-04-06_264short_05.avi
ffmpeg -ss 00:01:45 -i C:\Users\BMLab21\Documents\CrabStream

In [13]:
directory = r'C:\Users\BMLab21\Documents\CrabStreams\2022-04-06_264_clips'
video_name = '2022-04-06_264short_01.avi'
path = os.path.join(directory, video_name)
vid = pims.PyAVReaderIndexed(path)
vid

<Frames>
Source: C:\Users\BMLab21\Documents\CrabStreams\2022-04-06_264_clips\2022-04-06_264short_01.avi
Length: 170 frames
Frame Shape: (1080, 1920, 3)

In [33]:
time1 = time.time()
v_array = np.array(vid)
mean = np.mean(v_array, axis=3)
time2 = time.time()
print(time2-time1)

9.992537021636963


In [34]:
mean.shape

(170, 1080, 1920)

In [35]:
time1 = time.time()
cap = cv2.VideoCapture(path)
mean = []
while(True):
    ret, frame = cap.read()
    if ret == True:
        img = np.asarray(frame)
        mean.append(np.mean(img, axis=2))
    else:
        break

cap.release()
cv2.destroyAllWindows()
mean = np.array(mean)
time2 = time.time()
print(time2-time1)

9.838150024414062


In [36]:
mean.shape

(170, 1080, 1920)